<a href="https://colab.research.google.com/github/anupa35/Machine-Learning/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2



# 1. Preprocessing

In [54]:
import pandas as pd
import numpy as np

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
df = pd.read_csv("/content/drive/MyDrive/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [57]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [58]:
df.info() #checking data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [59]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') # changing incorrect data type of TotalCharges
df.isnull().sum() # checking for null values

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [60]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median()) # Handling missing values of TotalCharges

In [61]:
df.isnull().sum() # checking for null values again

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [62]:
df = df.drop(columns=['customerID']) # removing the customerID column since it is not useful

In [63]:
# converting Yes/No containing columns into 1/0 values
columnswith_YesNo = ['Partner', 'Dependents', 'PhoneService','PaperlessBilling', 'Churn']

for col in columnswith_YesNo:
    df[col] = df[col].map({'Yes':1, 'No':0})

In [64]:
# converting gender column data into 1/0
df['gender'] = df['gender'].map({'Male':1, 'Female':0})

In [65]:
# target
X = df.drop('Churn', axis=1)
y = df['Churn']

In [66]:
# splitting into training and testing
#from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [68]:
columns_multicatego = ['InternetService', 'Contract', 'PaymentMethod', 'MultipleLines','OnlineSecurity', 'OnlineBackup',
                       'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies']

In [69]:
columnswith_numeric = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [70]:
# For neural network
# using OneHotEncoding for columns containing multiple categories and scaling numeric columns

preprocessor_nn = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), columns_multicatego),
    ('numeric', StandardScaler(), columnswith_numeric)
])

In [71]:
# For Decision Tree
# using OneHotEncoding for columns containing multiple categories, no scaling
preprocessor_tree = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), columns_multicatego),
    ('numeric', 'passthrough', columnswith_numeric)  # no scaling
])

In [72]:
# For Neural network
# Fitting and transforming data

X_train_nn = preprocessor_nn.fit_transform(X_train)
X_test_nn = preprocessor_nn.transform(X_test)


In [73]:
# For Decision Tree
# Fitting and transforming data

X_train_tree = preprocessor_tree.fit_transform(X_train)
X_test_tree = preprocessor_tree.transform(X_test)

# Feed Forward Neural Network

In [74]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [75]:
# Compute balanced class weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

In [76]:
class_weight = {
    0: class_weights_array[0],
    1: class_weights_array[1]
}

print("Computed class weights:", class_weight)


Computed class weights: {0: np.float64(0.6805991785455424), 1: np.float64(1.8842809364548494)}


In [77]:
input_dim = X_train_nn.shape[1]

# Tuned neural network architecture
model = Sequential([
    Dense(128, activation='relu', input_dim=input_dim),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
# Tuned optimizer with custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [79]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

In [80]:
# Model training
history = model.fit(
    X_train_nn,
    y_train,
    validation_split=0.2,
    epochs=80,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weight,
    verbose=1
)

Epoch 1/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.6524 - loss: 0.6459 - val_accuracy: 0.7001 - val_loss: 0.5825
Epoch 2/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7073 - loss: 0.5356 - val_accuracy: 0.6841 - val_loss: 0.5293
Epoch 3/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7283 - loss: 0.5184 - val_accuracy: 0.7187 - val_loss: 0.5175
Epoch 4/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7281 - loss: 0.5251 - val_accuracy: 0.7161 - val_loss: 0.5159
Epoch 5/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7319 - loss: 0.5087 - val_accuracy: 0.7116 - val_loss: 0.5364
Epoch 6/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7282 - loss: 0.5103 - val_accuracy: 0.7169 - val_loss: 0.5337
Epoch 7/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7318 - loss: 0.4851 - val_accuracy: 0.7232 - val_loss: 0.5316
Epoch 8/80
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7372 - loss: 0.4847 - val_accuracy: 

In [91]:
# Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Confusion Matrix:
[[786 249]
 [ 91 283]]


In [90]:
# Predictions
y_pred_prob = model.predict(X_test_nn)
y_pred = (y_pred_prob > 0.55).astype(int)


print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nROC AUC Score:", roc_auc_score(y_test, y_pred_prob))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.76      0.82      1035
           1       0.53      0.76      0.62       374

    accuracy                           0.76      1409
   macro avg       0.71      0.76      0.72      1409
weighted avg       0.80      0.76      0.77      1409


ROC AUC Score: 0.8349414864760134


# Decision Tree